In [2]:
import pandas as pd
import datetime as datetime
import os
# ! pip install xlsxwriter --user

In [3]:
data_path=r'D:\PythonDev\MyQuantFinProject\Data'
data_file='Tech-World-Asia-Asea.csv'
isPy=False 
isPriceExcel=False

# Specify Period and Timefame to caculate fund performance and

In [4]:
from_str='2023-07-01'
to_str='2023-10-31' 
timeframe='M'
# timeframe='W-FRI'

# Input data

In [5]:
if isPy:
    print(f"Data csv file in Path: {data_path}")
    data_file=input("CSV file name : ")

    from_str=input("From_date(2020-10-01) : ") 
    to_str=input("To_date(2022-12-31): ") 
    timeframe=input("Timeframe(W-FRI,M,Q,Y): ") 
  
    print(f"Process data from {from_str} to {to_str} in {timeframe} timeframe")
    press_y=input(f"Press y=True and n=False to run analaystics : ") 
    if press_y.lower()=='y':
     ok=True
    else:
     exit()


In [6]:
report_prefix_name=data_file.split('.')[0]
data_file=os.path.join(data_path,data_file)
if os.path.exists(data_file)==False:
    raise Exception(f"not found {data_file}")
else:
    print(f"Process file {data_file}")
print(data_file)

Process file D:\PythonDev\MyQuantFinProject\Data\Tech-World-Asia-Asea.csv
D:\PythonDev\MyQuantFinProject\Data\Tech-World-Asia-Asea.csv


# Read csv file and extract and transform data as need

In [7]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.sort_index(inplace=True)
df=df.rename(columns={'Ticker':'symbol','close':'price'})
df=df.loc[from_str:to_str,['symbol','price']]
print(df.info())


# df=df.query("symbol!='^TNX'")



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 166 entries, 2023-07-03 to 2023-10-27
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  166 non-null    object 
 1   price   166 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.9+ KB
None


In [8]:
list_symbol=df['symbol'].unique().tolist() 

print(list_symbol)


['ACWI', 'QQQ']


In [9]:
df.head(len(list_symbol)*3)

,symbol,price
date,,
2023-07-03,ACWI,96.17
2023-07-03,QQQ,370.29
2023-07-05,QQQ,370.28
2023-07-05,ACWI,95.70
2023-07-06,ACWI,94.52
2023-07-06,QQQ,367.46


In [10]:
df.tail(len(list_symbol)*3)

,symbol,price
date,,
2023-10-25,QQQ,350.34
2023-10-25,ACWI,89.83
2023-10-26,QQQ,343.66
2023-10-26,ACWI,88.91
2023-10-27,ACWI,88.59
2023-10-27,QQQ,345.31


# Basic

# Resample data and calculate Percentage of change

In [11]:
colsROC=['symbol','date','pct_change']
df_Pct_Price=pd.DataFrame(columns=colsROC)
print(df_Pct_Price)

Empty DataFrame
Columns: [symbol, date, pct_change]
Index: []


In [14]:
#https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.aggregate.html
#https://stackoverflow.com/questions/47938372/pandas-dataframe-resample-aggregate-function-use-multiple-columns-with-a-customi
#https://www.alpharithms.com/aggregating-time-series-data-with-pandas-resampling-411212/
for symbol_name in list_symbol:

    dfx_symbol=df.query('symbol==@symbol_name')
    print(f"===================================={symbol_name}====================================")

    dfx_last=dfx_symbol.resample(timeframe).agg({'price': 'last'})
    dfx_std=dfx_symbol.resample(timeframe).agg({'price': 'std'})
    dfx_std=dfx_std.rename(columns={'price':'std'})
    print(dfx_last)
    print(dfx_std)
    dfx_px=pd.merge(left=dfx_last,right=dfx_std,how='inner',left_index=True, right_index=True)
    dfx_px=dfx_px.reset_index()
    print(dfx_px)
    

#     dfx_px['pct_change']=dfx_px['price'].pct_change()*100
#     dfx_px['pct_change']=dfx_px['pct_change'].round(2)

#     dfx_px=dfx_px.dropna() # the first  rown 
#     dfx_px=dfx_px.drop(columns=['price']) # remove price becase we are focuse ROC of Price
#     dfx_px=dfx_px[colsROC]
    
#     print(f"Calculate ROC of {symbol_name}")
#     print(dfx_px)
    
#     df_Pct_Price=pd.concat([df_Pct_Price,dfx_px],ignore_index=True,axis=0)

====================================ACWI====================================
            price
date             
2023-07-31  99.39
2023-08-31  96.50
2023-09-30  92.37
2023-10-31  88.59
                 std
date                
2023-07-31  1.542873
2023-08-31  1.359417
2023-09-30  1.597878
2023-10-31  1.604996
        date  price       std
0 2023-07-31  99.39  1.542873
1 2023-08-31  96.50  1.359417
2 2023-09-30  92.37  1.597878
3 2023-10-31  88.59  1.604996
====================================QQQ====================================
             price
date              
2023-07-31  383.68
2023-08-31  377.99
2023-09-30  358.27
2023-10-31  345.31
                 std
date                
2023-07-31  6.385399
2023-08-31  6.426087
2023-09-30  8.459875
2023-10-31  7.969677
        date   price       std
0 2023-07-31  383.68  6.385399
1 2023-08-31  377.99  6.426087
2 2023-09-30  358.27  8.459875
3 2023-10-31  345.31  7.969677


In [ ]:
print("==============================================================")
print("List calculate Percentage of change")
print(df_Pct_Price)

# Sum percentage of fund return

In [70]:
print("==============================================================")
print("Sum percentage of fund return")
dfSum_Pct_Price=df_Pct_Price.groupby(['symbol'])[['pct_change']].sum()
dfSum_Pct_Price.reset_index(drop=False,inplace=True)
dfSum_Pct_Price=dfSum_Pct_Price.sort_values(by="pct_change",ascending=False)
dfSum_Pct_Price["pct_change"]=dfSum_Pct_Price["pct_change"].round(1)
print(dfSum_Pct_Price)

Sum percentage of fund return
Empty DataFrame
Columns: [symbol, pct_change]
Index: []


In [71]:
# writer=pd.ExcelWriter( "BasicFundPerformanceReport.xlsx",engine='openpyxl') 

# df_Pct_Price.to_csv(f"ETF-{timeframe}-Return.csv",index=False)
# dfSum_Pct_Price.to_csv(f"ETF-{timeframe}-TotalReturn.csv",index=False)


# Advance

# Pivot Percent Return To Dataframe

In [64]:
print("==============================================================")
print("Pivot Percent Return To Dataframe")
dfPct=df_Pct_Price.pivot(index='date', columns='symbol', values='pct_change')
# print(dfPct.info())
print(dfPct)

symbol,ASP-SME,K-MIDSMALL,KKP-SM-CAP,KKP-TQG,KTMSEQ,TISCOMS-A
date,,,,,,
2023-09-08,-0.93,-0.11,-0.53,-1.04,0.05,0.54
2023-09-15,-0.94,-1.21,-0.53,0.12,-0.63,-0.83
2023-09-22,-0.69,-1.16,-1.07,0.00,-0.80,-0.98
2023-09-29,-2.02,-3.59,-2.29,-3.14,-2.84,-2.53


# Take Percent Return to Find Rank

In [65]:
print("==============================================================")
print("Take Percent Return to Find Rank")
dfRank=dfPct[dfPct.columns.tolist()].rank(method='max',axis=1)
print(dfRank)

symbol,ASP-SME,K-MIDSMALL,KKP-SM-CAP,KKP-TQG,KTMSEQ,TISCOMS-A
date,,,,,,
2023-09-08,2.0,4.0,3.0,1.0,5.0,6.0
2023-09-15,2.0,1.0,5.0,6.0,4.0,3.0
2023-09-22,5.0,1.0,2.0,6.0,4.0,3.0
2023-09-29,6.0,1.0,5.0,2.0,3.0,4.0


# Combine Both return and Rank

In [67]:
print("==============================================================")
print("Combine Both return and Rank")
dfSummary=pd.concat([\
                     dfPct.sum().to_frame(name='sum_return'),\
                     dfRank.sum().to_frame(name='sum_rank'),\
                    ],axis=1)
dfSummary.fillna(value=0,inplace=True)
print(dfSummary)

,sum_return,sum_rank
symbol,,
ASP-SME,-4.58,15.0
K-MIDSMALL,-6.07,7.0
KKP-SM-CAP,-4.42,15.0
KKP-TQG,-4.06,15.0
KTMSEQ,-4.22,16.0
TISCOMS-A,-3.80,16.0


# Summarize Total Score 

In [68]:
print("==============================================================")
print("Summarize Total Score ")
dfFinalScore=dfSummary.rank(method='max',axis=0)
dfFinalScore['TotalScore']=dfFinalScore.sum(axis=1)
dfFinalScore.sort_values(by='TotalScore',ascending=False,inplace=True)
print(dfFinalScore)

,sum_return,sum_rank,TotalScore
symbol,,,
TISCOMS-A,6.0,6.0,12.0
KTMSEQ,4.0,6.0,10.0
KKP-TQG,5.0,4.0,9.0
KKP-SM-CAP,3.0,4.0,7.0
ASP-SME,2.0,4.0,6.0
K-MIDSMALL,1.0,1.0,2.0


In [69]:
if isPriceExcel:
    print("Export outcoome as Excel")
    with pd.ExcelWriter(f"{report_prefix_name}_{timeframe}_{from_str}To{to_str}.xlsx") as writer:
        dfPct.to_excel(writer, sheet_name=f"{timeframe}-Return")
        dfSum_Pct_Price.to_excel(writer, sheet_name="Return")
        dfRank.to_excel(writer, sheet_name="RANK")
        dfSummary.to_excel(writer, sheet_name="Summary")
        dfFinalScore.to_excel(writer, sheet_name="FinalScore")